In [1]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy
import os

app = Flask(__name__)
app.config['SECRET_KEY'] = 'key_for_csrf_token_required_for_form_data'

db_path = os.path.abspath("synthetic_data/teleCC.sqlite")
app.config['SQLALCHEMY_DATABASE_URI'] = f"sqlite:///{db_path}"

# Track modifications is often unnecessary and can be disabled
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

In [2]:
from sqlalchemy import inspect

with app.app_context():
    inspector = inspect(db.engine)

    for table in inspector.get_table_names():
        print(table)


patient_teleCC
patient_teleCC_history
patient_teleCC_state
station_lookup


In [3]:
with app.app_context(): 
    db.session.expire_all()

In [4]:
from orm_models import  Patient_telecc_history, Patient_telecc_state
from flask import jsonify


In [25]:
def get_patient_details(patient_id):
    """
    Endpoint to fetch detailed information about a specific patient.
    """
    
    # Query the database for the patient details based on patient_id
    with app.app_context():
        patient = db.session.query(Patient_telecc_state).filter(Patient_telecc_state.patientID == patient_id).first()

        patient_hist = (
            db.session.query(Patient_telecc_history).
            filter(Patient_telecc_history.patientID == patient_id).
            order_by(Patient_telecc_history.prediction_datetime.asc()).
            all()
        )

        if not patient:
            return jsonify({'error': 'Patient not found'}), 404

        # Prepare the response data
        response_data = {
            'patient_id': patient.patientID,
            'patient_name': patient.patientName,
            'room': patient.room,
            'prediction_datetime': patient.prediction_datetime.strftime('%m-%d-%Y %I:%M %p'),
            'septic_shock_class': 'low' if patient.septic_shock_class == 0 else 'moderate' if patient.septic_shock_class == 1 else 'critical',
            'septic_shock_level': 'Low' if patient.septic_shock_class == 0 else 'Moderate' if patient.septic_shock_class == 1 else 'Critical',
            'septic_shock_protocol': 'Line 1\nLine 2\nLine 3\nLine 4',

            'septic_shock_expl': {
                f'{prefix}_{i}': (
                        getattr(patient, f'septic_shock_expl_field_{prefix}_{i}', None),  # Dynamically construct field name
                        getattr(patient, f'septic_shock_expl_val_{prefix}_{i}', None)
                    )
                    for prefix in ['pos', 'neg']  # Iterate over prefixes
                    for i in range(1, 4)          # Iterate over indices
            },   

            'respiratory_class': 'low' if patient.respiratory_class == 0 else 'moderate' if patient.respiratory_class == 1 else 'critical',
            'respiratory_level': 'Low' if patient.respiratory_class == 0 else 'Moderate' if patient.respiratory_class == 1 else 'Critical',
            'respiratory_protocol': 'Line A\nLine B\nLine C\nLine D',

            'respiratory_expl': {
                f'{prefix}_{i}': (
                        getattr(patient, f'respiratory_expl_field_{prefix}_{i}', None),  # Dynamically construct field name
                        getattr(patient, f'respiratory_expl_val_{prefix}_{i}', None)
                    )
                    for prefix in ['pos', 'neg']  # Iterate over prefixes
                    for i in range(1, 4)          # Iterate over indices
            }, 

            'septic_shock_history': [{
                "dateTime": record.prediction_datetime.isoformat(),
                "value": round(record.septic_shock_score, 2)
                }
                for record in patient_hist
            ],
            'respiratory_history': [{
                "dateTime": record.prediction_datetime.isoformat(),
                "value": round(record.respiratory_score, 2)
                }
                for record in patient_hist
            ]


        }

        return jsonify(response_data)

In [26]:
patientJSON = get_patient_details(1)
patientJSON.json


{'patient_id': 1,
 'patient_name': 'Charlie Miller',
 'prediction_datetime': '12-03-2024 12:00 PM',
 'respiratory_class': 'critical',
 'respiratory_expl': {'neg_1': ['Variable - D', -0.4047],
  'neg_2': ['Variable - E', -0.2701],
  'neg_3': ['Variable - F', -0.2389],
  'pos_1': ['Variable - A', 0.5994],
  'pos_2': ['Variable - B', 0.2497],
  'pos_3': ['Variable - C', 0.0752]},
 'respiratory_history': [{'dateTime': '2024-12-02T12:00:00', 'value': 0.45},
  {'dateTime': '2024-12-02T14:00:00', 'value': 0.42},
  {'dateTime': '2024-12-02T18:00:00', 'value': 0.43},
  {'dateTime': '2024-12-02T20:00:00', 'value': 0.41},
  {'dateTime': '2024-12-02T22:00:00', 'value': 0.41},
  {'dateTime': '2024-12-03T00:00:00', 'value': 0.4},
  {'dateTime': '2024-12-03T02:00:00', 'value': 0.37},
  {'dateTime': '2024-12-03T04:00:00', 'value': 0.35},
  {'dateTime': '2024-12-03T06:00:00', 'value': 0.32},
  {'dateTime': '2024-12-03T08:00:00', 'value': 0.3},
  {'dateTime': '2024-12-03T10:00:00', 'value': 0.29},
  {'d

In [7]:
with app.app_context():
    patient_hist = db.session.query(Patient_telecc_history).filter(Patient_telecc_history.patientID == 1).all()

In [14]:
respiratory_history = [round(score.respiratory_score,2) for score in patient_hist]
septic_shock_history = [round(score.septic_shock_score,2) for score in patient_hist]
septic_shock_history

[0.16, 0.13, 0.14, 0.18, 0.19, 0.18, 0.21, 0.22, 0.22, 0.25, 0.23, 0.25]

In [11]:
{
    f'{prefix}_{i}': (
        getattr(patient, f'septic_shock_expl_field_{prefix}_{i}', None),  # Dynamically construct field name
        getattr(patient, f'septic_shock_expl_val_{prefix}_{i}', None)
    )
    for prefix in ['pos', 'neg']  # Iterate over prefixes
    for i in range(1, 4)          # Iterate over indices
}

{'pos_1': ('Variable - A', 0.4425),
 'pos_2': ('Variable - B', 0.3253),
 'pos_3': ('Variable - C', 0.0419),
 'neg_1': ('Variable - D', -0.3565),
 'neg_2': ('Variable - E', -0.31),
 'neg_3': ('Variable - F', -0.1316)}

In [17]:
with app.app_context():
    db.session.expire_all()

In [29]:
{
    f'pos_{i}': (
        getattr(patient, f'septic_shock_expl_field_pos_{i}', None),  # Dynamically construct field name
        getattr(patient, f'septic_shock_expl_val_pos_{i}', None)
    )
    for i in range(1, 4)
}


{'pos_1': ('Variable - A', 0.4425),
 'pos_2': ('Variable - B', 0.3253),
 'pos_3': ('Variable - C', 0.0419)}

In [34]:
patient.septic_shock_expl_val_neg

AttributeError: 'Patient_telecc_state' object has no attribute 'septic_shock_expl_var_neg_1'

In [18]:
sign = 'pos'
i = 1
var = f'septic_shock_expl_field_{sign}_{i}'

getattr(patient, var)

'Variable - A'

In [ ]:
patient.septic_shock_expl_field_pos_1

In [13]:
x = get_patient_details(1)

HISTORY LENGTH: 12


In [12]:
x.json

{'patient_id': 1,
 'patient_name': 'Charlie Miller',
 'prediction_datetime': '12-01-2024 12:00 AM',
 'respiratory_class': 'critical',
 'respiratory_level': 'Critical',
 'respiratory_protocol': 'Line A\nLine B\nLine C\nLine D',
 'room': 2,
 'septic_shock_class': 'critical',
 'septic_shock_explainability': [{'contribution': 'septic_shock_expl_val_pos1',
   'variable': 'septic_shock_expl_field_pos1'},
  {'contribution': 'septic_shock_expl_val_pos2',
   'variable': 'septic_shock_expl_field_pos2'},
  {'contribution': 'septic_shock_expl_val_pos3',
   'variable': 'septic_shock_expl_field_pos3'}],
 'septic_shock_level': 'Critical',
 'septic_shock_protocol': 'Line 1\nLine 2\nLine 3\nLine 4'}

In [5]:
from sqlalchemy import inspect
from web_app import app, db
from orm_models import Patient_telecc_state

with app.app_context():
    inspector = inspect(db.engine)

    print("Tables:\n*********************")
    for table in inspector.get_table_names():
        print(table)
    print("*******************")

Resolved database path: c:\Users\GregSchwartz\OneDrive - National Consulting Partners, LLC\TeleCC\teleCC_web_app\synthetic_data\teleCC.sqlite
Database exists: True
Tables:
*********************
patient_teleCC
patient_teleCC_history
patient_teleCC_state
*******************
Tables:
*********************
patient_teleCC
patient_teleCC_history
patient_teleCC_state
*******************


In [2]:
from web_app import app, db
from orm_models import Patient_telecc_state

# Specify the desired Sta6a value
sta6a_value = 123  # Replace with your target Sta6a value

orm_patient = Patient_telecc_state()

with app.app_context():
    patientList = orm_patient.query.filter_by(Sta6a = sta6a_value).all()
    print(type(patientList))
    for  patient in patientList:
        print(patient.PatientID)

with app.app_context():
    results = db.session.query(Patient_telecc_state).filter(Patient_telecc_state.Sta6a == sta6a_value).all()

# Process or display the results
# for record in results:
#     print(record)


<class 'list'>
1
2
3
4
5
34
37
38
39
40
41
42
53
54


In [13]:
len(results)

14

In [8]:
import os
db_path = os.path.abspath("synthetic_data/teleCC.sqlite")
print(db_path)  # Verify the resolved absolute path
print(os.path.exists(db_path))  # Check if the file exists


c:\Users\GregSchwartz\OneDrive - National Consulting Partners, LLC\TeleCC\teleCC_web_app\synthetic_data\teleCC.sqlite
True
